In [3]:
#pip install PyAudio

In [4]:
#pip install SpeechRecognition

In [6]:
import speech_recognition as sr

# Inicializar el recognizer
r = sr.Recognizer()

# Utilizar el micrófono como fuente de audio
with sr.Microphone() as source:
    print("Por favor, hable ahora...")
    # Ajustar la energía del recognizer al nivel de ruido ambiente
    r.adjust_for_ambient_noise(source)
    # Escuchar el audio
    audio = r.listen(source)
    
    try:
        # Usar Google Web Speech API para reconocer el audio
        text = r.recognize_google(audio, language='es-ES')
        print("Usted dijo: " + text)
    except sr.UnknownValueError:
        print("Lo siento, no pude entender el audio.")
    except sr.RequestError as e:
        print("Error con el servicio de reconocimiento de voz; {0}".format(e))


Por favor, hable ahora...
Lo siento, no pude entender el audio.


In [8]:
#pip install pyttsx3

In [10]:
import pyttsx3

# Inicializar el motor de síntesis de voz
engine = pyttsx3.init()

# Configurar propiedades de la voz
engine.setProperty('rate', 150)  # Velocidad de habla
engine.setProperty('volume', 1)  # Volumen (0.0 a 1.0)

# Configurar el texto a ser leído
text = "Hola, este es un ejemplo de síntesis de voz con Python."

# Pasar el texto al motor y reproducirlo
engine.say(text)
engine.runAndWait()


# Combinación de funcionalidades NLP

In [11]:
import speech_recognition as sr
import pyttsx3
import nltk
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# Inicializar reconocimiento de voz y síntesis de voz
recognizer = sr.Recognizer()
engine = pyttsx3.init()

# Función para escuchar y reconocer voz
def escuchar():
    with sr.Microphone() as source:
        print("Por favor, hable ahora...")
        recognizer.adjust_for_ambient_noise(source)
        audio = recognizer.listen(source)
        try:
            texto = recognizer.recognize_google(audio, language='es-ES')
            print("Usted dijo: " + texto)
            return texto
        except sr.UnknownValueError:
            print("Lo siento, no pude entender el audio.")
            return ""
        except sr.RequestError as e:
            print("Error con el servicio de reconocimiento de voz; {0}".format(e))
            return ""

# Función para sintetizar voz
def hablar(texto):
    engine.say(texto)
    engine.runAndWait()

# Corpus de ejemplo
corpus = [
    "Hola, ¿Cómo te puedo ayudar?",
    "Ofrecemos servicios de desarrollo web, branding y marketing digital.",
    "El costo de una página web depende de la complejidad del proyecto.",
    "Nuestro proceso incluye diseño, desarrollo y pruebas.",
    "Sí, ofrecemos soporte 24/7.",
]

# Función para preprocesar el texto
def preprocesar(texto):
    stop_words = set(stopwords.words('spanish'))
    tokens = nltk.word_tokenize(texto.lower())
    tokens = [word for word in tokens if word.isalpha() and word not in stop_words]
    return ' '.join(tokens)

# Procesar el corpus
corpus_preprocesado = [preprocesar(sent) for sent in corpus]

# Vectorizador TF-IDF
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(corpus_preprocesado)

# Función para encontrar la mejor respuesta
def obtener_respuesta(texto_usuario):
    texto_usuario_preprocesado = preprocesar(texto_usuario)
    Y = vectorizer.transform([texto_usuario_preprocesado])
    similitudes = cosine_similarity(Y, X)
    idx_respuesta = similitudes.argmax()
    return corpus[idx_respuesta]

# Main loop
while True:
    texto_usuario = escuchar()
    if texto_usuario.lower() == 'adiós':
        hablar("Adiós, que tengas un buen día.")
        break
    elif texto_usuario:
        respuesta = obtener_respuesta(texto_usuario)
        print("Chatbot: " + respuesta)
        hablar(respuesta)


Por favor, hable ahora...
Usted dijo: Hola
Chatbot: Hola, ¿Cómo te puedo ayudar?
Por favor, hable ahora...
Usted dijo: Qué servicios tiene
Chatbot: Ofrecemos servicios de desarrollo web, branding y marketing digital.
Por favor, hable ahora...
Usted dijo: en un soporte
Chatbot: Sí, ofrecemos soporte 24/7.
Por favor, hable ahora...
Usted dijo: qué procesos tienen
Chatbot: Hola, ¿Cómo te puedo ayudar?
Por favor, hable ahora...
Usted dijo: adiós
